# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [69]:
song_files = get_files("data/song_data")

In [70]:
filepath = "data/song_data/A/A/A/TRAAAAW128F429D538.json" 

In [71]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [74]:
song_data = (df["song_id"][0],df["title"][0],df["artist_id"][0],int(df["year"][0]),df["duration"][0],)
song_data

('SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0,
 218.93179000000001)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [75]:
#cur.execute(song_table_insert, song_data)
cur.execute("""INSERT INTO song(song_id,title,artist_id,year,duration)  VALUES (%s,%s,%s,%s,%s);""" , song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = (df["artist_id"][0],df["artist_name"][0],df["artist_location"][0],df["artist_latitude"][0],df["artist_longitude"][0],)
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [77]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [6]:
log_files = "data/log_data"

In [7]:
filepath = "data/log_data/2018/11/2018-11-10-events.json"

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Hoobastank,Logged In,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Say The Same,200,1541808927796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
1,Mark Knopfler,Logged In,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Why Aye Man,200,1541809168796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
2,Mogwai,Logged In,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",PUT,NextSong,1541013292796,132,We're No Here,200,1541809417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
3,The Casualties,Logged In,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Punx Unite,200,1541809758796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
4,None,Logged In,Cecilia,F,0,Owens,NaN,free,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1541032432796,424,None,200,1541813635796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [9]:
df = df[df["page"]=="NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Hoobastank,Logged In,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Say The Same,200,1541808927796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
1,Mark Knopfler,Logged In,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Why Aye Man,200,1541809168796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
2,Mogwai,Logged In,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",PUT,NextSong,1541013292796,132,We're No Here,200,1541809417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
3,The Casualties,Logged In,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",PUT,NextSong,1541013292796,132,Punx Unite,200,1541809758796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",96
5,The Living End,Logged In,Ryan,M,0,Smith,188.62975,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,433,Roll On (Album Version),200,1541822502796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [23]:
t= pd.to_datetime(df['ts'], unit='ms')
print(type(t))
time_data = []
for line in t:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.dayofweek])
time_data[0]      

<class 'pandas.core.series.Series'>


[Timestamp('2018-11-10 00:15:27.796000'), 0, 10, 45, 11, 2018, 5]

In [10]:
df["ts"] = pd.to_datetime(df['ts'], unit='ms')
t = pd.Series(df["ts"])
t.head()

0   2018-11-10 00:15:27.796
1   2018-11-10 00:19:28.796
2   2018-11-10 00:23:37.796
3   2018-11-10 00:29:18.796
5   2018-11-10 04:01:42.796
Name: ts, dtype: datetime64[ns]

In [16]:
##print(type(t))
xx = ((t),(t.dt.hour))
print(type(xx))
time_df = pd.DataFrame(list(xx), columns= ['time'])
time_df.head()

<class 'tuple'>


TypeError: invalid type promotion

In [43]:
time_data = (t.append(t.dt.hour).append(t.dt.day).append(t.dt.week).append(t.dt.month).append(t.dt.year).append(t.dt.weekday))
column_labels = ("start_time", "hour", "day", "week", "month", "year", "weekday")

AttributeError: 'Timestamp' object has no attribute 'dt'

In [17]:
print(type(time_data[0]))

NameError: name 'time_data' is not defined

In [33]:
time_df = pd.DataFrame(list(time_data), columns= list(column_labels))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,0 2018-11-10 00:15:27.796 1 2018-11-10 0...,0 0 1 0 2 0 3 0 5 4 6...,0 10 1 10 2 10 3 10 5 10 6...,0 45 1 45 2 45 3 45 5 45 6...,0 11 1 11 2 11 3 11 5 11 6...,0 2018 1 2018 2 2018 3 2018 5 ...,0 5 1 5 2 5 3 5 5 5 6 ...


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.